File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [ ]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [ ]:
from rudata.RuDataMethod import Account, FintoolReferenceData, EndOfDayOnExchanges, AccruedInterestOnDate, RUPriceHistory, ExchangeTree, MoexSecurities, CurrencyRate, HistoryStockBonds, HistoryStockShares, HistoryStockNdm, HistoryStockCcp
Acc = Account()

Getting token started
Getting token finished


In [3]:
month_year = REPORT_DATE.strftime("%m.%Y")
isin_file = f"ISIN_{month_year}.xlsx"
Prices = pd.read_excel(f"./data/Input/{isin_file}", names=["ISIN"]).dropna().set_index("ISIN")
Prices["Date"] = REPORT_DATE
Prices[["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price", 
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]] = np.nan

FileNotFoundError: [Errno 2] No such file or directory: './data/Input/ISIN_06.2025.xlsx'

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [ ]:
FintoolReferenceDataDF = FintoolReferenceData().df
FintoolReferenceDataDF.head()

FintoolReferenceData started
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
300
5
0
5
0
FintoolReferenceData finished. FintoolReferenceData shape (105280, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,2.0,2.0,2.0,None,2.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,S111,None,Старшие необеспеченные,4000,None,None,"МинФин РФ, государственные краткосрочные беску...",None,None,ГКО-21134,NaN,21134RMFS,None,МинФин,0.0,act/365,None,None,Дисконтный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Краткосрочные,None,1.0,RU,0.0,RUB,1000.0,NaN,0.0,0.0,Погашен,2.500000e+09,2500000.0,2.500000e+09,2500000.0,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,None,0.0,1999-12-17T00:00:00,None,110.0,1999-12-17T00:00:00,2000-04-05T00:00:00,0.0,1999-12-17T00:00:00,Гос,Классические,Постоянный,2023-04-19T15:55:00,NaN,Нет информации,0.0,None,NaN,None,NaN,False,False,False,2.0,NaN,True,None,0.0,False,False,None,RU,1000,1000,NaN,False,NaN,False,False,None,416.0,NaN,None,None,105280,1
1,3.0,3.0,3.0,4.0,3.0,SU21135RMFS7,3.0,1.0,2.0,3.0,2.0,3.0,2.0,Государственные краткосрочные бескупонные обли...,Финансовые органы,None,135,0.0,0.0,None,0.0,0.0,1000.0,RUB,1500000.0,21135RMFS,1999-12-10T00:00:00,Финансовые орга

In [ ]:
%%time
EndOfDayOnExchangesDF = EndOfDayOnExchanges().df
EndOfDayOnExchangesDF.shape

EndOfDay started
88005
88005
5
0
5
0


KeyboardInterrupt: 

In [ ]:
AccruedInterestOnDateDF = AccruedInterestOnDate().df
AccruedInterestOnDateDF.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (104860, 15)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate,report_date
0,4000,0,Calendar,<NA>,2025-06-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,<NA>,RUB,RUB,2025-06-30T00:00:00,2025-06-20
1,4001,0,Calendar,<NA>,2025-06-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,<NA>,RUB,RUB,2025-06-30T00:00:00,2025-06-20
2,4002,0,Calendar,<NA>,2025-06-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,<NA>,RUB,RUB,2025-06-30T00:00:00,2025-06-20
3,4003,0,Calendar,<NA>,2025-06-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,<NA>,RUB,RUB,2025-06-30T00:00:00,2025-06-20
4,4004,0,Calendar,<NA>,2025-06-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,<NA>,RUB,RUB,2025-06-30T00:00:00,2025-06-20


In [ ]:
RUPriceHistoryDF = RUPriceHistory().df
RUPriceHistoryDF.shape

RUPriceHistory started
RUPriceHistory finished. RUPriceHistory shape (39082, 83)


(39082, 83)

In [ ]:
ExchangeTree = ExchangeTree().df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (242, 11)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn,report_date
0,1,0,РТС,RTS,Московская Биржа - РТС,RTS,12,2020-12-28T17:03:09,242,17,2025-06-30
1,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,242,4,2025-06-30
2,8,1,ФОРТС,FORTS,FORTS,FORTS,8,2020-12-28T17:03:09,242,11,2025-06-30
3,9,0,"ПАО ""СПБ""",PJSC «SPB»,"ПАО ""СПБ""",PJSC «SPB»,20,2020-12-28T17:03:09,242,21,2025-06-30
4,11,0,МБ,MX,Московская Биржа,Moscow Exchange,11,2020-12-28T17:03:09,242,16,2025-06-30


In [ ]:
MoexSecuritiesDF = MoexSecurities().df
MoexSecuritiesDF.head()

MoexSecurities started
MoexSecurities finished. MoexSecurities shape (219923, 25)


,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn,report_date
0,3107707,A-RM,LIQB,Продажа обеспечения бирж.рынок - безадрес.,stock_mamc_liq,Безадресные сделки с ЦК,mamc,stock,<NA>,US00846U1016,ESXXXX,Акции / обыкновенные,1,<NA>,<NA>,<NA>,<NA>,<NA>,0.010,127649.0,<NA>,2021-11-01T05:02:21.84541,219923,101898,2025-06-30
1,3108752,A-RM,TQBD,Т+: Ин.Акции и ДР (USD) - безадрес.,stock_foreignshares_tplus_usd,Т+: Ин.Акции и ДР (USD) - безадрес.,foreignshares,stock,<NA>,US00846U1016,ESXXXX,Акции / обыкновенные,1,<NA>,USD,<NA>,<NA>,<NA>,0.010,127649.0,<NA>,2022-03-23T13:26:55.926646,219923,102942,2025-06-30
2,3108944,A-RM,PSRD,РЕПО с ЦК (USD) - адрес.,stock_ccp_ndm_usd,РЕПО с ЦК (USD) - адрес.,ccp,stock,<NA>,US00846U1016,ESXXXX,Акции / обыкновенные,1,<NA>,<NA>,<NA>,<NA>,<NA>,0.001,127649.0,<NA>,2021-11-01T05:06:58.847436,219923,103134,2025-06-30
3,3108134,A-RM,EQRP,РЕПО с ЦК 1 день - безадрес.,stock_ccp,РЕПО с ЦК 1 день (RUB) - безадрес.,ccp,stock,<NA>,US00846U1016,ESXXXX,Акции / обыкновенные,1,<NA>,<NA>,<NA>,<NA>,<NA>,0.010,127649.0,<NA>,2021-11-01T05:03:57.846069,219923,102325,2025-06-30
4,3108147,A-RM,PSRE,РЕПО с ЦК (EUR) - адрес.,stock_ccp_ndm_eur,РЕПО с ЦК (EUR) - адрес.,ccp,stock,<NA>,US00846U1016,ESXXXX,Акции / обыкновенные,1,<NA>,<NA>,<NA>,<NA>,<NA>,0.001,127649.0,<NA>,2021-11-01T05:04:01.846094,219923,102338,2025-06-30


In [ ]:
CurrencyRateDF = CurrencyRate().df
CurrencyRateDF.head()

CurrencyRate started
CurrencyRate finished. CurrencyRate shape (166, 5)


,rate,endOfDayRate,error,currency,report_date
0,19.842100,19.842100,<NA>,LTL,2025-06-30
1,2.411370,2.414030,<NA>,THB,2025-06-30
2,0.000000,0.000000,<NA>,CDF,2025-06-30
3,10.013800,10.021500,<NA>,HKD,2025-06-30
4,0.004816,0.004838,<NA>,IDR,2025-06-30


##### Шаг 1.1. RuData MOEX stage

In [ ]:
HistoryStockBondsDF = HistoryStockBonds().df
HistoryStockBondsDF.head()

HistoryStockBonds started
HistoryStockBonds finished. HistoryStockBonds shape (111218, 66)


,boardname,regnumber,isin,matdate,listname,facevalue,currencyid,volume,value,numtrades,prev,prevlegalcloseprice,openperiod,legalopenprice,legalcloseprice,closeperiod,waprice,trendclose,trendclspr,trendwap,trendwappr,openval,closeval,highbid,lowoffer,bid,offer,yieldatwap,yieldclose,accint,duration,marketprice,marketprice2,admittedquote,issuesize,mpvaltrd,mp2valtrd,admittedvalue,marketprice3,marketprice3tradesvalue,type,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Облигации - безадрес.,4-02-00534-R,RU000A102TK9,2029-01-21T00:00:00,3,7500.0,SUR,0,0.0,0,88.9,88.9,NaN,None,88.9,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,62.08,100.0,62.07,100.0,NaN,NaN,37.97,NaN,None,NaN,None,9000,0.0,0.0,None,NaN,0.0,corporate_bond,NaN,NaN,NaN,8.8,54.25,None,2024-12-23T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A102TK9,ЮСК02К,2025-06-02T00:00:00,1,NaN
1,Т+: Облигации - безадрес.,4-02-00534-R,RU000A102TK9,2029-01-21T00:00:00,3,7500.0,SUR,0,0.0,0,88.9,88.9,NaN,None,88.9,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,62.06,100.0,62.06,100.0,NaN,NaN,39.78,NaN,None,NaN,None,9000,0.0,0.0,None,NaN,0.0,corporate_bond,NaN,NaN,NaN,8.8,54.25,None,2024-12-23T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A102TK9,ЮСК02К,2025-06-03T00:00:00,1,NaN
2,Т+: Облигации - безадрес.,4-02-00534-R,RU000A102TK9,2029-01-21T00:00:00,3,7500.0,SUR,0,0.0,0,88.9,88.9,NaN,None,88.9,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,62.06,NaN,62.06,NaN,NaN,NaN,41.59,NaN,None,NaN,None,9000,0.0,0.0,None,NaN,0.0,corporate_bond,NaN,NaN,NaN,8.8,54.25,None,2024-12-23T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A102TK9,ЮСК02К,2025-06-04T00:00:00,1,NaN
3,Т+: Облигации - безадрес.,4-02-00534-R,RU000A102TK9,2029-01-21T00:00:00,3,7500.0,SUR,0,0.0,0,88.9,88.9,NaN,None,88.9,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,62.08,100.0,62.07,100.0,NaN,NaN,43.40,NaN,None,NaN,None,9000,0.0,0.0,None,NaN,0.0,corporate_bond,NaN,NaN,NaN,8.8,54.25,None,2024-12-23T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A102TK9,ЮСК02К,2025-06-05T00:00:00,1,NaN
4,Т+: Облигации - безадрес.,4-02-00534-R,RU000A102TK9,2029-01-21T00:00:00,3,7500.0,SUR,0,0.0,0,88.9,88.9,NaN,None,88.9,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,62.06,NaN,62.06,NaN,NaN,NaN,45.21,NaN,None,NaN,None,9000,0.0,0.0,None,NaN,0.0,corporate_bond,NaN,NaN,NaN,8.8,54.25,None,2024-12-23T00:00:00,None,NaN,NaN,None,RUB,stock,bonds,TQCB,2,NaN,NaN,NaN,NaN,RU000A102TK9,ЮСК02К,2025-06-06T00:00:00,1,NaN


In [ ]:
HistoryStockSharesDF = HistoryStockShares().df
HistoryStockSharesDF.head()

HistoryStockShares started
HistoryStockShares finished. HistoryStockShares shape (25288, 54)


,boardname,regnumber,isin,listname,facevalue,currencyid,prevlegalcloseprice,prev,legalopenprice,openperiod,closeperiod,openval,closeval,legalcloseprice,trendclose,trendclspr,highbid,bid,offer,lowoffer,waprice,trendwap,trendwappr,volume,marketprice,marketprice2,mp2valtrd,mpvaltrd,value,numtrades,issuesize,admittedquote,admittedvalue,monthlycapitalization,dailycapitalization,marketprice3,marketprice3tradesvalue,type,closeauctionprice,waval,lastprice,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Акции и ДР - безадрес.,1-06-10167-F,RU000A0J2QG8,3,0.01,SUR,14.30,14.30,None,NaN,None,1432.0,15675.0,14.25,-0.05,-0.35,14.59,14.22,14.25,14.18,14.26,-0.03,-0.21,50300,None,14.26,717240.0,0.0,717240.0,99,1150000000,None,None,1.651400e+10,1.639900e+10,14.26,717240.0,common_share,14.25,0.0,14.25,stock,shares,TQBR,2,14.35,14.23,14.32,14.25,BLNG,Белон ао,2025-06-13T00:00:00,1,NaN
1,Т+: Акции и ДР - безадрес.,1-06-10167-F,RU000A0J2QG8,3,0.01,SUR,14.25,14.25,None,NaN,None,4263.0,70550.0,14.11,-0.14,-0.98,14.28,14.11,14.14,14.06,14.18,-0.08,-0.56,142700,None,14.18,2023043.0,0.0,2023043.0,209,1150000000,None,None,1.651400e+10,1.630700e+10,14.18,2023043.0,common_share,14.11,0.0,14.11,stock,shares,TQBR,2,14.33,14.03,14.21,14.11,BLNG,Белон ао,2025-06-16T00:00:00,1,NaN
2,Т+: Акции и ДР - безадрес.,1-06-10167-F,RU000A0J2QG8,3,0.01,SUR,14.11,14.11,None,NaN,None,1408.0,4308.0,14.36,0.25,1.77,14.37,14.27,14.30,14.05,14.24,0.06,0.42,123100,None,14.24,1753315.0,0.0,1753315.0,219,1150000000,None,None,1.651400e+10,1.637600e+10,14.24,1753315.0,common_share,14.36,0.0,14.29,stock,shares,TQBR,2,14.36,14.08,14.08,14.36,BLNG,Белон ао,2025-06-17T00:00:00,1,NaN
3,Т+: Акции и ДР - безадрес.,1-06-10167-F,RU000A0J2QG8,3,0.01,SUR,14.36,14.36,None,14.50,None,1450.0,2852.0,14.26,-0.10,-0.70,14.73,14.22,14.25,14.11,14.33,0.09,0.63,334700,None,14.33,4797778.0,0.0,4797778.0,339,1150000000,None,None,1.651400e+10,1.647950e+10,14.33,4797778.0,common_share,14.26,0.0,14.26,stock,shares,TQBR,2,14.52,14.16,14.50,14.26,BLNG,Белон ао,2025-06-18T00:00:00,1,NaN
4,Т+: Акции и ДР - безадрес.,1-06-10167-F,RU000A0J2QG8,3,0.01,SUR,14.26,14.26,None,14.25,None,1425.0,1423.0,14.23,-0.03,-0.21,14.47,14.21,14.23,14.15,14.27,-0.06,-0.42,56800,None,14.27,810512.0,0.0,810512.0,102,1150000000,None,None,1.651400e+10,1.641050e+10,14.27,810512.0,common_share,14.23,0.0,14.24,stock,shares,TQBR,2,14.41,14.19,14.25,14.23,BLNG,Белон ао,2025-06-19T00:00:00,1,NaN


In [ ]:
HistoryStockNdmDF = HistoryStockNdm().df
HistoryStockNdmDF.head()

HistoryStockNdm started
HistoryStockNdm finished. HistoryStockNdm shape (8311, 47)


,boardname,closeval,currencyid,facevalue,isin,issuesize,listname,matdate,numtrades,openval,prev,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,РПС с ЦК: Облигации - адрес.,255000000.0,SUR,1000.0,RU000A10BNF8,15000000,2,2027-08-11T00:00:00,1,255000000.0,100.25,4B02-13-55038-E-001P,1.75,1.75,-1.38,-1.33,exchange_bond,255000000.0,250000,102.00,17.51,17.51,None,NaN,None,17.35,14.26,None,2025-06-16T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PTOB,2,102.00,102.00,102.00,102.00,RU000A10BNF8,РусГидБП13,2025-06-16T00:00:00,3,NaN
1,РПС: Облигации - адрес.,30540000.0,SUR,1000.0,RU000A10BNF8,15000000,2,2027-08-11T00:00:00,1,30540000.0,100.00,4B02-13-55038-E-001P,1.80,1.80,0.22,0.22,exchange_bond,30540000.0,30000,101.88,17.59,17.64,None,NaN,None,17.35,14.26,None,2025-06-10T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSOB,2,101.80,101.80,101.80,101.80,RU000A10BNF8,РусГидБП13,2025-06-10T00:00:00,1,NaN
2,РПС: Облигации - адрес.,307020000.0,SUR,1000.0,RU000A10BNF8,15000000,2,2027-08-11T00:00:00,1,307020000.0,101.80,4B02-13-55038-E-001P,0.20,0.20,-1.38,-1.33,exchange_bond,307020000.0,301000,102.00,17.51,17.51,None,NaN,None,17.35,14.26,None,2025-06-16T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PSOB,2,102.00,102.00,102.00,102.00,RU000A10BNF8,РусГидБП13,2025-06-16T00:00:00,1,NaN
3,РПС с ЦК: Облигации - адрес.,255000000.0,SUR,1000.0,RU000A10BNF8,15000000,2,2027-08-11T00:00:00,1,255000000.0,100.25,4B02-13-55038-E-001P,1.75,1.75,-1.38,-1.33,exchange_bond,255000000.0,250000,102.00,17.51,17.51,None,NaN,None,17.35,14.26,None,2025-06-16T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PTOB,2,102.00,102.00,102.00,102.00,RU000A10BNF8,РусГидБП13,2025-06-16T00:00:00,1,NaN
4,РПС с ЦК: Облигации - адрес.,25940000.0,SUR,1000.0,RU000A10ASS2,18000000,2,2026-05-29T00:00:00,2,25912500.0,103.10,4B02-14-00124-A-001P,0.66,0.64,0.06,0.06,exchange_bond,51852500.0,50000,103.75,18.76,18.76,None,NaN,None,21.65,17.79,None,2025-06-17T00:00:00,None,NaN,NaN,None,RUB,stock,ndm,PTOB,2,103.76,103.65,103.65,103.76,RU000A10ASS2,РостелP14R,2025-06-17T00:00:00,3,NaN


In [ ]:
HistoryStockCcpDF = HistoryStockCcp().df
HistoryStockCcDF.head()

HistoryStockCcp started
HistoryStockCcp finished. HistoryStockCcp shape (37220, 54)


,accint,admittedquote,admittedvalue,bid,boardname,closeperiod,closeval,currencyid,duration,facevalue,highbid,isin,issuesize,legalcloseprice,legalopenprice,listname,lowoffer,marketprice,marketprice2,marketprice3,marketprice3tradesvalue,matdate,mp2valtrd,mpvaltrd,numtrades,offer,openperiod,openval,prev,prevlegalcloseprice,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,7.91,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,4.273950e+09,SUR,NaN,1000.0,19.75,RU000A102A31,427747115,None,None,1,19.75,19.75,None,None,0.0,2031-11-26T00:00:00,0.0,0.0,2,None,None,4.956975e+09,19.80,None,29018RMFS,-0.05,None,-0.05,NaN,ofz_bond,9.230925e+09,10129121,19.75,None,None,stock,ccp,EQRP,4,19.75,19.75,19.75,19.75,SU29018RMFS7,ОФЗ 29018,2025-06-18T00:00:00,3,NaN
1,7.91,None,None,None,РЕПО с ЦК - адрес.,None,1.086960e+03,SUR,NaN,1000.0,NaN,RU000A102A31,427747115,None,None,1,NaN,19.75,None,None,0.0,2031-11-26T00:00:00,0.0,0.0,21,None,None,9.704900e+02,NaN,None,29018RMFS,NaN,None,NaN,NaN,ofz_bond,6.133501e+04,62,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29018RMFS7,ОФЗ 29018,2025-06-18T00:00:00,3,NaN
2,8.46,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.977505e+05,SUR,NaN,1000.0,19.90,RU000A102A31,427747115,None,None,1,19.50,19.79,None,None,0.0,2031-11-26T00:00:00,0.0,0.0,3,None,None,1.690447e+06,19.75,None,29018RMFS,-0.23,None,-0.13,NaN,ofz_bond,7.469709e+08,819683,19.62,None,None,stock,ccp,EQRP,4,19.90,19.52,19.90,19.52,SU29018RMFS7,ОФЗ 29018,2025-06-19T00:00:00,3,NaN
3,8.46,None,None,None,РЕПО с ЦК - адрес.,None,1.088690e+03,SUR,NaN,1000.0,NaN,RU000A102A31,427747115,None,None,1,NaN,19.79,None,None,0.0,2031-11-26T00:00:00,0.0,0.0,21,None,None,9.719600e+02,NaN,None,29018RMFS,NaN,None,NaN,NaN,ofz_bond,6.022322e+04,61,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29018RMFS7,ОФЗ 29018,2025-06-19T00:00:00,3,NaN
4,9.00,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.661059e+06,SUR,NaN,1000.0,19.90,RU000A102A31,427747115,None,None,1,19.90,19.81,None,None,0.0,2031-11-26T00:00:00,0.0,0.0,1,None,None,1.661059e+06,19.52,None,29018RMFS,0.38,None,0.28,NaN,ofz_bond,1.661059e+06,1820,19.90,None,None,stock,ccp,EQRP,4,19.90,19.90,19.90,19.90,SU29018RMFS7,ОФЗ 29018,2025-06-20T00:00:00,3,NaN


##### Шаг 1.1. Discounts

###### Шаг 1.1.1. CBR

In [15]:
from sources.CBR_Soap import CBR_Soap

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.6834.111 Safari/537.36', 
           'Accept-Encoding': 'gzip, deflate', 
           'Accept': '*/*', 
           'Connection': 'keep-alive'}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.set_index('ISIN')

GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.10


/Users/alexander/PycharmProjects/insurance_mine/sources/CBR_Soap.py:212: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  disc = pd.concat([disc, pd.DataFrame(index=[0], data=d)], axis=0)


In [16]:
Prices.update(cbr_disc)
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [17]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecurities[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()
currMarketRates = currMarketRates[currMarketRates.index.str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)]
currMarketRates=currMarketRates[['Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))) #Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates /= 100

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_4164/88371159.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)


In [18]:
Prices.update(currMarketRates)
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [19]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.set_index('ISIN')
securInfo = securInfo[securInfo['NCC collateral']=='Да']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_4164/1680612647.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\


In [20]:
Prices.update(securInfo)
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [15]:
TransformFintoolReferenceData = FintoolReferenceData[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type", 
    "faceftname": "Currency"
})
TransformFintoolReferenceData["ISIN in DB"] = 1
TransformFintoolReferenceData["Source"] = 'RU Data (ЦЦ НРД)'
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    AccruedInterestOnDate[['fintoolId', 'accruedInterest', 'currentFaceValue']], how="left", left_on="Fintool", right_on="fintoolId")
TransformFintoolReferenceData = TransformFintoolReferenceData\
.rename(columns={
    'accruedInterest': 'Interest', 
    'currentFaceValue': 'Principal'})\
.drop(columns=["fintoolId"])

In [16]:
RUPriceHistoryLast = RUPriceHistory.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()

In [17]:
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
TransformFintoolReferenceData["Interest"] = TransformFintoolReferenceData["Interest"].combine_first(TransformFintoolReferenceData["InterestHistory"])
TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)
TransformFintoolReferenceData.drop(columns="InterestHistory", inplace=True)

In [18]:
bonds_index = TransformFintoolReferenceData[TransformFintoolReferenceData["Type"]=='Облигация'].index

In [19]:
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"] + \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Interest"]

TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"]

In [20]:
Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())

NameError: name 'Prices' is not defined

In [21]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [32]:
EndOfDay.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 209 columns):
 #    Column              Non-Null Count  Dtype        
---   ------              --------------  -----        
 0    accruedint          907 non-null    float64      
 1    admquote            0 non-null      string       
 2    akm_bl_n            0 non-null      string       
 3    akm_e_n             0 non-null      string       
 4    ask                 993 non-null    float64      
 5    ask_size            0 non-null      string       
 6    ask_size_total      993 non-null    float64      
 7    avge_chng           923 non-null    float64      
 8    avge_chng_pct       923 non-null    float64      
 9    avge_prce           965 non-null    float64      
 10   avge_yield          873 non-null    float64      
 11   bid                 995 non-null    float64      
 12   bid_size            0 non-null      string       
 13   bid_size_total      993 non-null    float64   

In [ ]:
important_cols = [
    "isin", "seccode", "secname", "name", "fintoolId", "id_iss", "id_trade_site",
    "report_date", "add_date", "update_date", "mat_date", "last_time",
    "close", "last", "open", "high", "low", "mprice", "avge_prce", "bid", "ask",
    "vol", "val", "val_usd", "vol_lots", "deal_acc",
    "yield_bid", "yield_ask", "last_yield", "yield_agg", "yield_swa",
    "duration", "pvbp", "convexity", "spread",
    "boardid", "boardname", "exch", "currency"
]

In [36]:
EndOfDay['id_trade_site']

0       170
1       170
2       170
3       170
4       170
       ... 
1018    170
1019    170
1020    170
1021    170
1022    170
Name: id_trade_site, Length: 1023, dtype: Int64

In [25]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices

,fintoolId,last,fullname_rus
0,4200,66.3900,МБ Основной Т+
1,4226,1026.0000,МБ Основной Т+
2,4298,59.3000,МБ Основной Т+
3,4345,428.0000,МБ Основной Т+
4,4355,6252.5000,МБ Основной Т+
...,...,...,...
1018,250508,100.8500,МБ Основной Т+
1019,250533,100.9400,МБ Основной Т+
1020,250582,99.9900,МБ Основной Т+
1021,250698,101.4476,МБ Основной Т+


In [22]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_50587/1720937637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_50587/1720937637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_50587/1720937637.py:4: SettingWithCopyWarning: 
A value

In [23]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_50587/3772375966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_50587/3772375966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Price'] = exch_not_bonds['PriceEX']


In [24]:
exch_bonds

,Fintool,PriceEX,Source,ISIN,Type,Principal,Interest,Clean price pct,Clean price,Price
48,6043,104.7999,RU Data (МБ Основной Т+),XS0114288789,Облигация,0.05,0.000937,104.7999,0.0524,0.053337
49,6043,104.7999,RU Data (МБ Основной Т+),XS0114288789,Облигация,0.05,0.000937,104.7999,0.0524,0.053337
50,6044,120.4945,RU Data (МБ Основной Т+),XS0088543193,Облигация,1000.00,2.130000,120.4945,1204.9450,1207.075000
51,6044,120.4945,RU Data (МБ Основной Т+),XS0088543193,Облигация,1000.00,2.130000,120.4945,1204.9450,1207.075000
72,18936,61.3840,RU Data (МБ Основной Т+),RU000A0GN9A7,Облигация,1000.00,26.090000,61.3840,613.8400,639.930000
...,...,...,...,...,...,...,...,...,...,...
2041,250582,99.9900,RU Data (МБ Основной Т+),RU000A10BWC6,Облигация,1000.00,1.480000,99.9900,999.9000,1001.380000
2042,250698,101.4476,RU Data (МБ Основной Т+),RU000A10BV55,Облигация,1000.00,1.230000,101.4476,1014.4760,1015.706000
2043,250698,101.4476,RU Data (МБ Основной Т+),RU000A10BV55,Облигация,1000.00,1.230000,101.4476,1014.4760,1015.706000
2044,250962,99.8900,RU Data (МБ Основной Т+),RU000A10BW62,Облигация,1000.00,1.480000,99.8900,998.9000,1000.380000


In [30]:
Prices.update(exch_bonds.drop_duplicates().set_index("ISIN"))
Prices.update(exch_not_bonds.drop_duplicates().set_index("ISIN"))
# Prices.update(exch_not_bonds.set_index("ISIN"))

In [31]:
ResultPrices = Prices[~Prices['Price'].isna()]
Prices = Prices[Prices['Price'].isna()]

#### Шаг 2. Cbonds

##### Шаг 2.1. Bonds

In [32]:
tradings_bonds = pd.read_excel(f'./data/Input/tradings_bonds.xls')
tradings_bonds_1 = pd.read_excel(f'./data/Input/tradings_bonds_1.xls')

tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)
tradings_bonds = tradings_bonds[['ISIN', 'Краткое название эмитента (rus)', 'Валюта', 'Индикативная цена', 'Номинал по еврооблигациям']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта': 'Currency', 
    'Индикативная цена': 'Clean price pct', 
    'Номинал по еврооблигациям': 'Principal EB'
})

In [33]:
tradings_bonds = tradings_bonds.merge(Prices[["Interest", "Principal"]], how='inner', left_on="ISIN", right_index=True)
principal_idx = tradings_bonds[tradings_bonds['Principal']>tradings_bonds['Principal EB']].index
tradings_bonds.loc[principal_idx, 'Principal'] /= tradings_bonds.loc[principal_idx, 'Principal EB']
tradings_bonds.loc[principal_idx, 'Interest'] /= tradings_bonds.loc[principal_idx, 'Principal']

In [34]:
Prices.update(tradings_bonds.drop_duplicates("ISIN").set_index("ISIN"))

In [35]:
tradings_nkd = pd.read_excel(f'./data/Input/tradings_nkd.xls')
tradings_nkd_1 = pd.read_excel(f'./data/Input/tradings_nkd_1.xls')
tradings_nkd_2 = pd.read_excel(f'./data/Input/tradings_nkd_2.xls')
tradings_nkd_3 = pd.read_excel(f'./data/Input/tradings_nkd_3.xls')

tradings_nkd = pd.concat([tradings_nkd, tradings_nkd_1, tradings_nkd_2, tradings_nkd_3], axis=0)
tradings_nkd = tradings_nkd[['ISIN код эмиссии', 'Накопленный купонный доход', 'Непогашенный номинал']]\
.rename(columns={
    'ISIN код эмиссии': "ISIN", 
    'Накопленный купонный доход': "Interest", 
    'Непогашенный номинал': "Principal"
})
tradings_nkd = tradings_nkd.groupby("ISIN").first()

In [36]:
Prices.update(tradings_nkd)

In [37]:
tradings_new = pd.read_excel(f'./data/Input/tradings_new_{REPORT_DATE_STR}.xls')

tradings_new = tradings_new[['Код ISIN', 'Эмиссия', 'Дата', 'Bid, закр.', 'НКД']]\
.rename(columns={
    'Код ISIN': 'ISIN',
    'Эмиссия': 'Name',
    'Дата': 'Date',
    'Bid, закр.': 'Clean price pct',
    'НКД': 'Interest'
})
tradings_new["Max date"] = tradings_new.groupby('ISIN')["Date"].transform('max')
tradings_new = tradings_new.groupby('ISIN').first()

In [38]:
emissions = pd.read_excel(f'./data/Input/emissions_{REPORT_DATE_STR}.xls')

emissions = emissions[['Валюта', 'код ISIN', 'Непогашенный номинал']]\
.rename(columns={
    'код ISIN': 'ISIN',
    'Валюта': 'Currency',
    'Непогашенный номинал': 'Principal'
})
emissions = emissions[~((emissions["ISIN"].isna()) | (emissions["Principal"].isna()) | (emissions["Principal"]==0))].set_index("ISIN")
tradings_new = tradings_new.merge(emissions, how="left", left_index=True, right_index=True)

In [39]:
Prices.update(tradings_new)

In [40]:
Prices.loc[Prices['Interest'] > 0.2*Prices['Principal'], 'Interest'] = 0
Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
Prices['Price'] = Prices['Clean price'] + Prices['Interest']

In [41]:
cbonds_bonds_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[cbonds_bonds_idx, "Type"] = 'Облигация'
Prices.loc[cbonds_bonds_idx, 'Source'] = 'Cbonds'
Prices.loc[cbonds_bonds_idx, 'ISIN in DB'] = 1

In [42]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[cbonds_bonds_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

##### Шаг 2.2. Stocks

In [43]:
tradings_stocks_all = pd.read_excel(f'./data/Input/tradings_stocks_all.xls')

tradings_stocks_all = tradings_stocks_all[['ISIN', 'Краткое название эмитента (rus)', 'Валюта номинала', 'Валюта торгов', 'Цена закрытия']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта номинала': 'Currency', 
    'Валюта торгов': 'Trade currency', 
    'Цена закрытия': 'Price'
})

In [44]:
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Trade currency", right_on="currency")\
.rename(columns={'rate': 'trade rate'})
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Currency", right_on="currency")
tradings_stocks_all['RUB Price'] = tradings_stocks_all['Price'] * tradings_stocks_all['trade rate']
tradings_stocks_all['RUB Price'] = tradings_stocks_all.groupby("ISIN")['RUB Price'].transform('mean')
tradings_stocks_all['Currency'] = tradings_stocks_all['Currency'].combine_first(tradings_stocks_all['Trade currency'])
tradings_stocks_all['Clean price'] = tradings_stocks_all['RUB Price'] / tradings_stocks_all['rate']
tradings_stocks_all['Price'] = tradings_stocks_all['Clean price']
tradings_stocks_all['FX Rate'] = tradings_stocks_all['trade rate'].combine_first(tradings_stocks_all['rate'])

In [45]:
cbonds_stocks_idx = tradings_stocks_all[~tradings_stocks_all["Price"].isna()].index
tradings_stocks_all.loc[cbonds_stocks_idx, "Type"] = 'Акция'
tradings_stocks_all.loc[cbonds_stocks_idx, 'Source'] = 'Cbonds'
tradings_stocks_all.loc[cbonds_stocks_idx, 'ISIN in DB'] = 1

In [46]:
Prices.update(tradings_stocks_all.groupby("ISIN").first())

In [47]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices["Price"].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

#### Шаг 3. MOEX

##### Шаг 3.1. Tranform

In [52]:
TransformHistoryStockBonds = HistoryStockBonds.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockShares.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdm.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcp.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [53]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [54]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [55]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [56]:
Prices.shape

(1496, 16)

#### Шаг 4. Finish

In [57]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [58]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRate[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [59]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [60]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

In [61]:
FullPrices['export_timestamp'] = dt.now().replace(microsecond=0)
FullPrices['Date'] = last_day_month
FullPrices['Source'] = FullPrices['Source'].fillna(0)
FullPrices['ISIN in DB'] = FullPrices['ISIN in DB'].fillna(0)

In [62]:
with pd.ExcelWriter(f'./data/Output/prices.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices.to_excel(writer)

In [63]:
# with pd.ExcelWriter(f'./data/Output/prices_full.xlsx', datetime_format='dd.mm.yyyy') as writer: 
#     FullPrices.to_excel(writer)

In [64]:
from functions.send_email import main
main("prices.xlsx")